<a href="https://colab.research.google.com/github/Norrr3r/ET-ANER/blob/main/1_Test_for_Majority_and_Weighted_voting_CANER_Corpus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install seqeval==1.2.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 891.3 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=23fc809ceae3914a3f14eff91416502417eccfcc0a0de3a4687fba136cd10c86
  Stored in directory: /root/.cache/pip/wheels/5f/b8/73/0b2c1a76b701a677653dd79ece07cfabd7457989dbfbdcd8d7
Successfully built seqeval


In [2]:
import os
from sklearn.metrics import accuracy_score
#from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score



In [3]:
from seqeval.metrics import (
    classification_report,
    precision_score,
    recall_score,
    f1_score as f1,
    accuracy_score)
from seqeval.scheme import IOB2


In [4]:
def combine_predictions_for_hard_voting (files):
 #out = open("out.txt", "r+")
 bst='\t'
 list1 = [[] for h in range(25950)]
 dic ={}


 i=0  #index for files
 for file in files:
    #index = 3
    with open( file, 'r', encoding='utf-8') as f:
        k=0   #index for lines
        for line in f:

            if not line.strip():
              list1[k].append([''])

            else:
                line = line.strip().split()
                token, gold_tag,predicted_tag = line[0], line[1],line[2]
                if i==0:
                   list1[k].append([token])
                   list1[k].append([gold_tag])
                   list1[k].append([predicted_tag])
                #else:
                 #   list1[k].append([predicted_tag])

                elif any(token in sublist for sublist in list1[k]):
                   list1[k].append([predicted_tag])
                else:
                   dic[token] = predicted_tag

            k+=1
    i+=1

 for token, tag in dic.items():
       for i, sublist in enumerate(list1):
            if any(token in sublist for sublist in list1[i]):
               list1[i].append([tag])
               break


 with open('/content/Predictions_test.txt', 'w', encoding='utf-8') as f:
    for kline in list1:
        for item in kline:
            f.write( bst.join(item) + '\t')
        f.write('\n')
#    for token , ptags in dic.items():
 #    f.write( f"{token} \t {ptags} \n")

 file_name = '/content/Predictions_test.txt'
 return file_name



In [5]:
def Hard_Voting (files):
 predicted_tags = []
 vote_counts = {}
 list2=[]
 file_name = combine_predictions_for_hard_voting(files)
 with open(file_name, "r", encoding="utf-8") as f:
   for line in f:
            line = line.strip().split()

            if len(line)==0:
              continue

            elif len(line)==3:
              token = line[0]
              gold_tag = line[1]
              pred_tag = line[2]
              list2.append((token ,gold_tag, pred_tag))

            elif len(line) >3:
              token = line[0]
              gold_tag = line[1]

              for tag in line[2:]:
                if tag not in vote_counts:
                   vote_counts[tag] = 0
                vote_counts[tag] += 1
              pred_tag = max(vote_counts, key=vote_counts.get)
              list2.append((token ,gold_tag, pred_tag))
              vote_counts.clear()


 with open('Compined_Predictions.txt', 'w', encoding='utf-8') as f:
    for token, gold_tag, pred_tag in list2:
        f.write(f"{token}\t{gold_tag}\t{pred_tag}\n")

 file_voting = 'Compined_Predictions.txt'

 return file_voting

In [6]:
def Allmetrics_hard_voting (files):
 for file in files:
  with open(file, 'r') as f:
    data = f.read().strip().split('\n')

  sentences_gold_tag = []
  sentences_pred_tag = []
  gold_tags = []
  pred_tags = []

  for line in data[1:]:
    if not line.strip():
        # Add the current sentence's gold and pred tags to the list of all sentences
        sentences_gold_tag.append(gold_tags)
        sentences_pred_tag.append(pred_tags)
        # Reset the gold_tags and pred_tags lists for the next sentence
        gold_tags = []
        pred_tags = []
    else:
        token, gold_tag, pred_tag = line.strip().split('\t')
        gold_tags.append(gold_tag)
        pred_tags.append(pred_tag)

# Add the gold and pred tags for the last sentence to the list of all sentences
  sentences_gold_tag.append(gold_tags)
  sentences_pred_tag.append(pred_tags)

# Compute precision, recall, and F1-score
  precision= precision_score(sentences_gold_tag, sentences_pred_tag, average='micro' , scheme=IOB2)
  recall= recall_score(sentences_gold_tag, sentences_pred_tag, average='micro' , scheme=IOB2)
  f1_score= f1(sentences_gold_tag, sentences_pred_tag, average='micro', scheme=IOB2 )
  report = classification_report(sentences_gold_tag, sentences_pred_tag, scheme=IOB2, digits=4)



# Print the results
  print ('\n*************\n', file )

  print(f"Precision: {precision:.3f}")
  print(f"Recall: {recall:.3f}")
  print(f"F1-score: {f1_score:.3f}")

  #print (report)

#Hard Voting for test set

In [7]:

New_files = [
          "/content/Camel_msa_predictions.txt",
       "/content/Camel_ca_predictions.txt",
         "/content/Camel_mix_predictions.txt",
      "/content/multilingual_predictions.txt",
         ]


file_vote_predictions = Hard_Voting(New_files)

New_files.append(file_vote_predictions)


Allmetrics_hard_voting(New_files)

/usr/local/lib/python3.12/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: Allah seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.12/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: Clan seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.12/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: Prophet seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.12/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: Pers seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.12/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: NatOb seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.12/dist-packages/seqeval/metrics/sequence_labeling.p


*************
 /content/Camel_msa_predictions.txt
Precision: 0.968
Recall: 0.977
F1-score: 0.973

*************
 /content/Camel_ca_predictions.txt
Precision: 0.975
Recall: 0.981
F1-score: 0.978

*************
 /content/Camel_mix_predictions.txt
Precision: 0.969
Recall: 0.978
F1-score: 0.974

*************
 /content/multilingual_predictions.txt
Precision: 0.961
Recall: 0.969
F1-score: 0.965

*************
 Compined_Predictions.txt
Precision: 0.973
Recall: 0.981
F1-score: 0.977


#Soft Voting

In [8]:
def combine_files_for_soft_voting (files):
 list1 = [[] for h in range(25950)]
 dic ={}


 i=0
 for file , weight in files:
    #index = 3
    with open(file, 'r', encoding='utf-8') as f:
        k=0
        for line in f:

            if not line.strip():
              list1[k].append([''])

            else:
                line = line.strip().split()
                token, gold_tag,predicted_tag = line[0], line[1],line[2]
                if i==0:
                   list1[k].append([token])
                   list1[k].append([gold_tag])
                   list1[k].append([predicted_tag , weight])

                elif any(token in sublist for sublist in list1[k]):
                   list1[k].append([predicted_tag , weight])
                else:
                   dic[token] = [predicted_tag, weight]

            k+=1
    i+=1

 for token, tag in dic.items():
        for i, sublist in enumerate(list1):
          if any(isinstance(sublist, list) and token in sublist for sublist in list1[i]):
               #if isinstance(tag, list):
               # list1[i].extend(tag)
               #else:
                list1[i].append(tag)
                break


 with open('Predictions_soft_hard.txt', 'w', encoding='utf-8') as f:
    for kline in list1:
        for item in kline:
          if isinstance(item, list):
            f.write(','.join(map(str, item)))
          else:
            f.write(str(item))
          f.write('\t')
        f.write('\n')
#    for token , ptags in dic.items():
 #    f.write( f"{token} \t {ptags} \n")

 file_name = '/content/Predictions_soft_hard.txt'

 return file_name




In [9]:
def Soft_voting(files):
 model_weight =[]
 predicted_tags = []
 vote_counts = {}
 list2=[]
 label_sums = {}
 file_name = combine_files_for_soft_voting(files)

 with open(file_name, "r", encoding="utf-8") as f:
   for line in f:

            line = line.strip().split()

            if len(line)==0:
              continue
            elif len(line) == 3:
              token = line[0]
              gold_tag = line[1]
              pred_tag = line[2]
              list2.append((token , gold_tag, pred_tag))

            elif len(line) > 3:
              token = line[0]
              gold_tag = line[1]
              for tag in line[2:]:
                values = tag.split(',')
                if len(values) !=2:
                  continue
                label , value = values
                if label not in vote_counts:
                   vote_counts[label] = 0
                vote_counts[label] += float(value)
              pred_tag = max(vote_counts, key=vote_counts.get)
              list2.append((token , gold_tag, pred_tag))
              vote_counts.clear()


 with open('Compined_Predictions_soft_hard.txt', 'w', encoding='utf-8') as f:
    for token, gold_tag, pred_tag in list2:
        f.write(f"{token}\t{gold_tag}\t{pred_tag}\n")

 file_name = '/content/Compined_Predictions_soft_hard.txt'

 return file_name

In [13]:
def Allmetrics_soft_voting (files):

 for file , weight in files:
  with open(file, 'r', encoding='utf-8') as f:
    data = f.read().strip().split('\n')

  sentences_gold_tag = []
  sentences_pred_tag = []
  gold_tags = []
  pred_tags = []

  for line in data[1:]:
    if not line.strip():
        # Add the current sentence's gold and pred tags to the list of all sentences
        sentences_gold_tag.append(gold_tags)
        sentences_pred_tag.append(pred_tags)
        # Reset the gold_tags and pred_tags lists for the next sentence
        gold_tags = []
        pred_tags = []
    else:
        token, gold_tag, pred_tag = line.strip().split('\t')
        gold_tags.append(gold_tag)
        pred_tags.append(pred_tag)

# Add the gold and pred tags for the last sentence to the list of all sentences
  sentences_gold_tag.append(gold_tags)
  sentences_pred_tag.append(pred_tags)

# Compute precision, recall, and F1-score
  precision= precision_score(sentences_gold_tag, sentences_pred_tag, average='micro' , scheme=IOB2)
  recall= recall_score(sentences_gold_tag, sentences_pred_tag, average='micro' , scheme=IOB2)
  f1_score= f1(sentences_gold_tag, sentences_pred_tag, average='micro', scheme=IOB2 )
  report = classification_report(sentences_gold_tag, sentences_pred_tag, scheme=IOB2, digits=4)



# Print the results
  print ('\n*************\n', file )

  print(f"Precision: {precision:.4f}")
  print(f"Recall: {recall:.4f}")
  print(f"F1-score: {f1_score:.4f}")

  #print (report)

In [17]:
#weighted for each model

files = [
         [ "/content/Camel_msa_predictions.txt", 0.249487], #929
        ["/content/Camel_ca_predictions.txt", 0.250769],  #88
        ["/content/Camel_mix_predictions.txt", 0.249744],  #924
        ["/content/multilingual_predictions.txt",  0.25]  #936
         ]


file_vote_predictions = Soft_voting(files)

files.append([file_vote_predictions,0.0])


Allmetrics_soft_voting(files)



/usr/local/lib/python3.12/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: Allah seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.12/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: Clan seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.12/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: Prophet seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.12/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: Pers seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.12/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: NatOb seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.12/dist-packages/seqeval/metrics/sequence_labeling.p


*************
 /content/Camel_msa_predictions.txt
Precision: 0.9685
Recall: 0.9775
F1-score: 0.9730

*************
 /content/Camel_ca_predictions.txt
Precision: 0.9751
Recall: 0.9805
F1-score: 0.9778

*************
 /content/Camel_mix_predictions.txt
Precision: 0.9694
Recall: 0.9784
F1-score: 0.9739

*************
 /content/multilingual_predictions.txt
Precision: 0.9611
Recall: 0.9694
F1-score: 0.9653

*************
 /content/Compined_Predictions_soft_hard.txt
Precision: 0.9754
Recall: 0.9805
F1-score: 0.9780
